# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [3]:
'''
1.	Недостатки текущей реализации:
	1	Поддерживаются только операции сложения и умножения.
	2	Отсутствует поддержка унарных операций (например, отрицание).
	3	Нет поддержки деления и возведения в степень.
	4	Ограничена поддержка функций одной переменной.
2.	Добавление унарных операций, деления и возведения в степень:

Для этого добавим методы для унарных операций (__neg__), деления (__truediv__), и возведения в степень (__pow__).

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value ** 2))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                raise NotImplementedError("Поддержка возведения в степень с Dual как показателем не реализована")
            case Number():
                return Dual(self.value ** float(other), float(other) * (self.value ** (float(other) - 1)) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    __rmul__ = __mul__
    __radd__ = __add__
    __rtruediv__ = lambda self, other: Dual(float(other), 0).__truediv__(self)  # поддержка деления вида "2 / x"
'''

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [4]:
'''
import math

@dataclass
class Dual:
    value: float
    d: float

    # ... предыдущие методы ...

    def exp(self) -> "Dual":
        return Dual(math.exp(self.value), math.exp(self.value) * self.d)

    def sin(self) -> "Dual":
        return Dual(math.sin(self.value), math.cos(self.value) * self.d)

    def cos(self) -> "Dual":
        return Dual(math.cos(self.value), -math.sin(self.value) * self.d)

    def log(self) -> "Dual":
        return Dual(math.log(self.value), self.d / self.value)
'''
'''
def test_functions():
    x = Dual(1, 1)
    
    # Проверка exp
    assert round(x.exp().value, 5) == round(math.exp(1), 5)
    
    # Проверка sin и cos
    assert round(x.sin().value, 5) == round(math.sin(1), 5)
    assert round(x.cos().value, 5) == round(math.cos(1), 5)
    
    # Проверка log
    assert round(x.log().value, 5) == round(math.log(1), 5)

test_functions()
'''

## Задание 1.3 (2 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+1 балл**)

In [5]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [6]:
'''
Для численной проверки воспользуемся библиотекой scipy:
from scipy.misc import derivative

def test_numerical_derivative():
    f = lambda x: 5 * x * x + 2 * x + 2
    f_diff = diff(f)
    
    assert round(f_diff(2), 5) == round(derivative(f, 2.0), 5)

test_numerical_derivative()
'''

## Задание 1.4 (6 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
'''
Можно использовать стандартный модуль ast для генерации случайных функций и их проверки.

Пример генерации функции и проверки её производной:
import ast
import random

def random_function():
    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[ast.arg(arg='x')],
                posonlyargs=[],
                kwonlyargs=[], kw_defaults=[], defaults=[]
            ),
            body=ast.BinOp(
                left=ast.BinOp(
                    left=ast.Constant(value=random.randint(1, 10)),
                    op=ast.Mult(),
                    right=ast.Name(id='x', ctx=ast.Load())
                ),
                op=random.choice([ast.Add(), ast.Sub()]),
                right=ast.Constant(value=random.randint(1, 10))
            )
        )
    )
    ast.fix_missing_locations(expr)
    return eval(compile(expr, filename="", mode="eval"))

def test_random_functions():
    for _ in range(10):
        func = random_function()
        f_diff = diff(func)
        x = random.uniform(0, 10)
        assert round(f_diff(x), 5) == round(derivative(func, x), 5)

test_random_functions()
'''

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
'''
Чтобы поддерживать функции с несколькими аргументами, нужно адаптировать систему дифференцирования:
def diff_multivariate(func: Callable[..., float]) -> Callable[..., list[float]]:
    def wrapper(*args: float) -> list[float]:
        gradients = []
        for i in range(len(args)):
            dual_args = [Dual(arg, 1.0 if i == j else 0.0) for j, arg in enumerate(args)]
            gradients.append(func(*dual_args).d)
        return gradients
    return wrapper

def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y

f_diff = diff_multivariate(f)

# Проверка
assert f_diff(10, 10, 10) == [10, 5, 1]
Теперь система поддерживает многомерные функции и может возвращать градиенты для каждого аргумента.
'''